In [96]:
import pandas as pd
import numpy as np
import re
import glob
import os

folder_path = '/Volumes/Extreme SSD/SET_Intern/SET/SOMC/MAY_Greek/Option Greeks (05-2025)'
os.chdir(folder_path)

In [ ]:
def process_file(file_path):
    """
    Args:
        file_path (str): The full path to the Excel file.

    Returns:
        pandas.DataFrame: A DataFrame containing the extracted data.
    """
    month_order = {
    'F': 1, 'G': 2, 'H': 3, 'J': 4, 'K': 5, 'M': 6, 
    'N': 7, 'Q': 8, 'U': 9, 'V': 10, 'X': 11, 'Z': 12
    }
    month_to_code_map = {value: key for key, value in month_order.items()}
    
    df = pd.read_csv(file_path, header=0)
    df['Date'] = file_path.split('/')[-1].split('_')[0].split('.')[0]
    df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y', errors='coerce')
    df['Call/Put'] = df['Call/Put'].apply(lambda x: 'C' if str.lower(x) == 'call' else 'P')
    df['month_code'] = df['Contract Month'].str.split('/').str[1].astype(int)
    df['month_code'] = df['month_code'].map(month_to_code_map)
    df['year_code'] = df['Contract Month'].str.split('/').str[2].str.replace('20', '').astype(str)
    df['Product'] = 'S50' + df['month_code'] + df['year_code'] + df['Call/Put'] + df['Strike Price'].astype(str)
    df = df[['Date', 'Product', 'Delta', 'Gamma', 'Vega', 'Rho', 'Theta']]
    
    return df

In [ ]:
# --- Main part of the script ---
csv_files = glob.glob(os.path.join(folder_path, '*.csv'))

if not csv_files:
    print(f"No csv files (.csv) found in the folder: {folder_path}")
else:
    print(f"Found {len(csv_files)} csv files to process.")

    all_dataframes = [] # List to hold all DataFrames

    for file in csv_files:
        if not os.path.basename(file).startswith('~'):
            df_single = process_file(file)
            if df_single is not None:
                all_dataframes.append(df_single)

    if all_dataframes:
        merged_df = pd.concat(all_dataframes, ignore_index=True)

        # Remove duplicate rows
        rows_before_dedupe = len(merged_df)
        merged_df = merged_df.drop_duplicates(keep='first', ignore_index=True) # <<< NEW LINE
        rows_after_dedupe = len(merged_df)
        print(f"Removed {rows_before_dedupe - rows_after_dedupe} duplicate rows.")
        
        # Display results
        print("\n--- Merged DataFrame (Cleaned) ---")
        print(f"Total rows: {len(merged_df)}")
        print("First 5 rows:")
        print(merged_df.head())
        print("\nLast 5 rows:")
        print(merged_df.tail())

Found 19 csv files to process.
Removed 0 duplicate rows.

--- Merged DataFrame (Cleaned) ---
Total rows: 2244
First 5 rows:
        Date     Product     Delta     Gamma       Vega        Rho      Theta
0 2025-05-02  S50K25C575  0.997221  0.000001   0.008102  42.483122  19.791530
1 2025-05-02  S50K25C600  0.997120  0.000008   0.091488  44.324300  19.242391
2 2025-05-02  S50K25C625  0.996305  0.000057   0.662355  46.123623  17.831349
3 2025-05-02  S50K25C650  0.991913  0.000280   3.226132  47.712435  12.886072
4 2025-05-02  S50K25C675  0.975378  0.000957  11.022036  48.582774  -1.381997

Last 5 rows:
           Date     Product     Delta     Gamma        Vega         Rho  \
2239 2025-05-30  S50U25P810 -0.733968  0.003390  137.082646 -210.723882   
2240 2025-05-30  S50U25P825 -0.778070  0.003048  123.268179 -225.726443   
2241 2025-05-30  S50U25P850 -0.839343  0.002450   99.086796 -248.084178   
2242 2025-05-30  S50U25P875 -0.886075  0.001878   75.941983 -267.211633   
2243 2025-05-30  S5

In [99]:
delta = pd.pivot_table(merged_df, index='Date', columns='Product', values='Delta')
delta

Product,S50K25C575,S50K25C600,S50K25C625,S50K25C650,S50K25C675,S50K25C700,S50K25C725,S50K25C750,S50K25C775,S50K25C800,...,S50U25P770,S50U25P775,S50U25P780,S50U25P790,S50U25P800,S50U25P810,S50U25P825,S50U25P850,S50U25P875,S50U25P900
Date,,,,,,,,,,,,,,,,,,,,,
2025-05-02,0.997221,0.997120,0.996305,0.991913,0.975378,0.930292,0.838419,0.694608,0.517402,0.341734,...,NaN,-0.474969,NaN,NaN,-0.550153,NaN,-0.621049,-0.685782,-0.743175,NaN
2025-05-06,0.997612,0.997568,0.997072,0.993635,0.978074,0.930083,0.825446,0.658790,0.459194,0.274742,...,NaN,-0.499404,NaN,NaN,-0.576251,NaN,-0.647726,-0.712001,-0.768053,NaN
2025-05-07,0.997790,0.997781,0.997658,0.996654,0.991190,0.970706,0.915849,0.807540,0.645418,0.456801,...,NaN,-0.423683,NaN,NaN,-0.497258,NaN,-0.568444,-0.635171,-0.695941,NaN
2025-05-08,0.997861,0.997849,0.997679,0.996261,0.988611,0.960705,0.889273,0.756609,0.572503,0.376514,...,NaN,-0.453938,NaN,NaN,-0.528650,NaN,-0.599859,-0.665594,-0.724531,-0.775985
2025-05-09,0.997980,0.997977,0.997916,0.997267,0.992927,0.973992,0.917814,0.800290,0.621314,0.417178,...,NaN,-0.436619,NaN,NaN,-0.512768,NaN,-0.585891,-0.653788,-0.714933,-0.768479
2025-05-13,0.998384,0.998384,0.998378,0.998252,0.996768,0.986670,0.944875,0.834698,0.641963,0.409962,...,NaN,-0.434587,NaN,NaN,-0.513428,NaN,-0.589096,-0.659146,-0.721898,-0.776450
2025-05-14,0.998493,0.998493,0.998492,0.998451,0.997792,0.991929,0.961889,0.868460,0.683924,0.443004,...,NaN,-0.421886,NaN,NaN,-0.502339,NaN,-0.579980,-0.652123,-0.716878,-0.773198
2025-05-15,0.998567,0.998566,0.998562,0.998427,0.996543,0.982527,0.923359,0.773972,0.537638,0.293521,...,NaN,-0.471903,NaN,NaN,-0.551943,NaN,-0.627251,-0.695559,-0.755487,-0.806486
2025-05-16,0.998669,0.998669,0.998668,0.998614,0.997578,0.987656,0.937239,0.792929,0.547872,0.289438,...,NaN,-0.470128,NaN,NaN,-0.552121,NaN,-0.629207,-0.698945,-0.759862,-0.811400


In [100]:
gamma = pd.pivot_table(merged_df, index='Date', columns='Product', values='Gamma')
gamma

Product,S50K25C575,S50K25C600,S50K25C625,S50K25C650,S50K25C675,S50K25C700,S50K25C725,S50K25C750,S50K25C775,S50K25C800,...,S50U25P770,S50U25P775,S50U25P780,S50U25P790,S50U25P800,S50U25P810,S50U25P825,S50U25P850,S50U25P875,S50U25P900
Date,,,,,,,,,,,,,,,,,,,,,
2025-05-02,0.000001,0.000008,0.000057,0.000280,0.000957,0.002378,0.004437,0.006392,0.007289,0.006722,...,NaN,0.003054,NaN,NaN,0.003024,NaN,0.002891,0.002678,0.002410,NaN
2025-05-06,0.000000,0.000004,0.000039,0.000240,0.000965,0.002654,0.005183,0.007430,0.008053,0.006774,...,NaN,0.003171,NaN,NaN,0.003099,NaN,0.002921,0.002663,0.002355,NaN
2025-05-07,0.000000,0.000001,0.000010,0.000073,0.000354,0.001197,0.002899,0.005198,0.007103,0.007583,...,NaN,0.002889,NaN,NaN,0.002934,NaN,0.002879,0.002739,0.002533,NaN
2025-05-08,0.000000,0.000001,0.000014,0.000104,0.000499,0.001625,0.003730,0.006239,0.007838,0.007595,...,NaN,0.002993,NaN,NaN,0.002995,NaN,0.002895,0.002712,0.002468,0.002188
2025-05-09,0.000000,0.000000,0.000005,0.000051,0.000302,0.001169,0.003083,0.005755,0.007856,0.008073,...,NaN,0.003015,NaN,NaN,0.003043,NaN,0.002961,0.002788,0.002546,0.002263
2025-05-13,0.000000,0.000000,0.000001,0.000012,0.000123,0.000722,0.002570,0.005830,0.008797,0.009171,...,NaN,0.003118,NaN,NaN,0.003148,NaN,0.003058,0.002867,0.002603,0.002294
2025-05-14,0.000000,0.000000,0.000000,0.000004,0.000060,0.000455,0.001982,0.005229,0.008771,0.009752,...,NaN,0.003155,NaN,NaN,0.003207,NaN,0.003129,0.002942,0.002674,0.002356
2025-05-15,0.000000,0.000000,0.000001,0.000014,0.000165,0.001036,0.003663,0.007734,0.010242,0.008884,...,NaN,0.003243,NaN,NaN,0.003212,NaN,0.003057,0.002807,0.002495,0.002152
2025-05-16,0.000000,0.000000,0.000000,0.000006,0.000100,0.000792,0.003315,0.007777,0.010806,0.009339,...,NaN,0.003318,NaN,NaN,0.003286,NaN,0.003123,0.002858,0.002527,0.002165


In [101]:
Vega = pd.pivot_table(merged_df, index='Date', columns='Product', values='Vega')
Vega

Product,S50K25C575,S50K25C600,S50K25C625,S50K25C650,S50K25C675,S50K25C700,S50K25C725,S50K25C750,S50K25C775,S50K25C800,...,S50U25P770,S50U25P775,S50U25P780,S50U25P790,S50U25P800,S50U25P810,S50U25P825,S50U25P850,S50U25P875,S50U25P900
Date,,,,,,,,,,,,,,,,,,,,,
2025-05-02,0.008102,0.091488,0.662355,3.226132,11.022036,27.390995,51.111816,73.642112,83.968368,77.438362,...,NaN,195.456614,NaN,NaN,193.526723,NaN,185.053067,171.423069,154.265708,NaN
2025-05-06,0.002225,0.037451,0.370230,2.277666,9.167528,25.216249,49.240636,70.585804,76.503282,64.351423,...,NaN,191.221536,NaN,NaN,186.888166,NaN,176.122387,160.565926,142.025436,NaN
2025-05-07,0.000439,0.008574,0.100052,0.737571,3.604122,12.173487,29.483215,52.877672,72.252532,77.134680,...,NaN,193.649001,NaN,NaN,196.680032,NaN,193.037594,183.644854,169.804880,NaN
2025-05-08,0.000525,0.010889,0.131140,0.972929,4.678916,15.246727,34.995670,58.539977,73.542987,71.258174,...,NaN,192.553543,NaN,NaN,192.686712,NaN,186.264044,174.470491,158.791641,140.773008
2025-05-09,0.000102,0.002980,0.048011,0.455901,2.696676,10.427946,27.500146,51.329257,70.067406,72.008896,...,NaN,192.296954,NaN,NaN,194.035048,NaN,188.817530,177.770063,162.398093,144.321774
2025-05-13,0.000001,0.000119,0.004488,0.084721,0.863390,5.057936,17.996893,40.821087,61.597761,64.216564,...,NaN,189.649327,NaN,NaN,191.511115,NaN,186.021143,174.403984,158.313429,139.524174
2025-05-14,0.000000,0.000017,0.001002,0.027882,0.390952,2.962600,12.897516,34.036273,57.089264,63.469088,...,NaN,188.906962,NaN,NaN,192.009795,NaN,187.361859,176.154641,160.091619,141.047755
2025-05-15,0.000001,0.000065,0.003455,0.082718,0.976669,6.113907,21.618142,45.646427,60.449675,52.437928,...,NaN,187.330656,NaN,NaN,185.493543,NaN,176.572499,162.151842,144.109760,124.298043
2025-05-16,0.000000,0.000011,0.000924,0.033111,0.536394,4.262378,17.837957,41.848331,58.152285,50.255532,...,NaN,186.805663,NaN,NaN,185.013172,NaN,175.807965,160.880623,142.244045,121.875687


In [102]:
Rho = pd.pivot_table(merged_df, index='Date', columns='Product', values='Rho')
Rho

Product,S50K25C575,S50K25C600,S50K25C625,S50K25C650,S50K25C675,S50K25C700,S50K25C725,S50K25C750,S50K25C775,S50K25C800,...,S50U25P770,S50U25P775,S50U25P780,S50U25P790,S50U25P800,S50U25P810,S50U25P825,S50U25P850,S50U25P875,S50U25P900
Date,,,,,,,,,,,,,,,,,,,,,
2025-05-02,42.483122,44.324300,46.123623,47.712435,48.582774,47.750502,44.092007,37.214718,28.106313,18.753414,...,NaN,-173.450529,NaN,NaN,-203.435353,NaN,-232.782349,-260.819042,-287.086996,NaN
2025-05-06,36.196280,37.767848,39.317146,40.720788,41.521173,40.695979,36.995044,30.058729,21.223286,12.815593,...,NaN,-175.845339,NaN,NaN,-205.554294,NaN,-234.325457,-261.514818,-286.718417,NaN
2025-05-07,34.624584,36.129547,37.629025,39.085478,40.323640,40.822640,39.616302,35.720511,29.033175,20.802835,...,NaN,-152.027205,NaN,NaN,-180.501978,NaN,-208.938731,-236.630449,-263.035927,NaN
2025-05-08,33.052290,34.488773,35.918043,37.289983,38.374338,38.518773,36.624600,31.812638,24.440350,16.250231,...,NaN,-160.170794,NaN,NaN,-188.805085,NaN,-217.063962,-244.273357,-269.949024,-293.799449
2025-05-09,31.479732,32.848270,34.214299,35.554418,36.731586,37.261231,36.125753,32.209948,25.420531,17.269920,...,NaN,-153.301380,NaN,NaN,-182.173915,NaN,-210.834193,-238.541164,-264.747957,-289.112147
2025-05-13,25.188086,26.283216,27.378148,28.468871,29.511810,30.250483,29.863134,27.020952,21.144321,13.664491,...,NaN,-147.722473,NaN,NaN,-176.586133,NaN,-205.223726,-232.837402,-258.844550,-282.889893
2025-05-14,23.614810,24.641540,25.668230,26.693629,27.698533,28.530702,28.556249,26.448737,21.218379,13.918701,...,NaN,-142.437404,NaN,NaN,-171.567156,NaN,-200.597288,-228.660047,-255.111356,-279.550438
2025-05-15,22.041493,22.999816,23.957997,24.912240,25.813495,26.345572,25.494934,21.846719,15.407214,8.495992,...,NaN,-156.319459,NaN,NaN,-185.123079,NaN,-213.255670,-239.980905,-264.809617,-287.493158
2025-05-16,20.467982,21.357894,22.247770,23.136178,23.996943,24.606913,24.067362,20.833224,14.620136,7.801769,...,NaN,-154.235480,NaN,NaN,-183.398532,NaN,-211.861125,-238.838140,-263.813313,-286.532080


In [103]:
Theta = pd.pivot_table(merged_df, index='Date', columns='Product', values='Theta')
Theta

Product,S50K25C575,S50K25C600,S50K25C625,S50K25C650,S50K25C675,S50K25C700,S50K25C725,S50K25C750,S50K25C775,S50K25C800,...,S50U25P770,S50U25P775,S50U25P780,S50U25P790,S50U25P800,S50U25P810,S50U25P825,S50U25P850,S50U25P875,S50U25P900
Date,,,,,,,,,,,,,,,,,,,,,
2025-05-02,19.791530,19.242391,17.831349,12.886072,-1.381997,-31.063665,-74.315698,-116.303860,-137.494106,-129.187127,...,NaN,-68.403631,NaN,NaN,-68.813361,NaN,-67.068831,-63.577271,-58.833809,NaN
2025-05-06,19.910347,19.439229,18.359605,14.050675,-0.522253,-34.127592,-84.764714,-130.978532,-146.510759,-125.213629,...,NaN,-68.382474,NaN,NaN,-68.057764,NaN,-65.567177,-61.405468,-56.133794,NaN
2025-05-07,20.004862,19.592733,18.993557,17.166591,10.313737,-9.455403,-49.201473,-103.321648,-149.347092,-163.536551,...,NaN,-71.594117,NaN,NaN,-73.628810,NaN,-73.345493,-71.023213,-67.079074,NaN
2025-05-08,20.065263,19.648896,18.976295,16.617697,7.545449,-17.697477,-64.842418,-121.735419,-159.716231,-157.920643,...,NaN,-71.665980,NaN,NaN,-72.748878,NaN,-71.538114,-68.401811,-63.820686,-58.306560
2025-05-09,20.042514,19.643142,19.142339,17.766854,11.961462,-7.171290,-49.177797,-108.280674,-156.214023,-164.451006,...,NaN,-70.679934,NaN,NaN,-72.319575,NaN,-71.560649,-68.744110,-64.349293,-58.905423
2025-05-13,20.054401,19.660647,19.254391,18.624656,15.933371,3.127380,-35.727911,-104.555307,-168.721345,-180.442907,...,NaN,-70.294299,NaN,NaN,-72.024676,NaN,-71.201213,-68.197196,-63.543531,-57.825034
2025-05-14,20.019737,19.625230,19.227742,18.750446,17.235175,8.882593,-22.375533,-88.913484,-162.758405,-186.477853,...,NaN,-69.118709,NaN,NaN,-71.277434,NaN,-70.782804,-67.988660,-63.432979,-57.724976
2025-05-15,20.032212,19.639013,19.234729,18.577086,15.193231,-2.439959,-55.140795,-137.540022,-190.786160,-168.535843,...,NaN,-70.561996,NaN,NaN,-71.063640,NaN,-69.059131,-65.049313,-59.649848,-53.481669
2025-05-16,20.063746,19.670686,19.274449,18.768083,16.599540,3.024495,-45.631987,-132.288001,-193.642446,-170.575617,...,NaN,-69.569717,NaN,NaN,-70.129448,NaN,-68.100406,-64.009142,-58.506711,-52.247881


## Excel Writer

In [ ]:
sheet_dict = {
    'Delta': delta,
    'Gamma': gamma,
    'Vega': Vega,
    'Rho': Rho,
    'Theta': Theta
}
output_filename = 'MAY_Greek_Data.xlsx'
os.chdir('/Users/fulinq/Documents/SET_Intern/SOMC/Greek') # Folder path to save the output

with pd.ExcelWriter(output_filename) as writer:
    for sheet_name, data in sheet_dict.items():
        data.to_excel(writer, sheet_name=sheet_name)